In [30]:
import networkExpansionPy.lib as ne
import numpy as np
import pandas as pd
import pickle as pickle
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import spearmanr,pearsonr,mannwhitneyu
from statsmodels.stats.multitest import multipletests
plt.rcParams['font.family'] = 'Arial'

import warnings
from scipy.sparse import (spdiags, SparseEfficiencyWarning, csc_matrix,
    csr_matrix, isspmatrix, dok_matrix, lil_matrix, bsr_matrix)
warnings.simplefilter('ignore',SparseEfficiencyWarning)

def load_data(network_name):
    with open(ne.asset_path  + "/metabolic_networks/" + network_name,"rb") as filehandler:
        metabolism = pickle.load(filehandler)
    return metabolism

# load the metabolism object from the repo
#metabolism = load_data('metabolism.23Aug2022.pkl')
#model_id = "metabolism.v4.20Mar2023.pkl"
#model_id = "metabolism.v5.12Apr2023.pkl"
model_id = "metabolism.v8.01May2023.pkl"
metabolism = pd.read_pickle('/Users/joshuagoldford/Documents/github/network-expansion-assets/models/' + model_id)
cids = metabolism.coenzymes

In [31]:
# run expansion
seed_set = pd.read_csv('../assets/seed_set.csv')['ID'].tolist()

#seed_df = pd.read_csv('assets/seed_set.csv')
#seed_set = seed_df[~seed_df.Type.isin(["Organic carbon"])].ID.tolist()
#seed_set = seed_set + ["C00010"]

print('There were {N} compounds in this seed set...'.format(N=len(seed_set)))

ne_cpds,ne_rxns = metabolism.expand(seed_set,algorithm='trace')
print('There were {N} compounds produced in this expansion!'.format(N=len(ne_cpds)))

# parse data into dataframes
rn_df = pd.DataFrame(pd.Series(ne_rxns),columns = ['iteration'])
rn_df.reset_index(inplace=True)
rn_df.columns = ['rn','direction','iteration']
rn_df['rn_kegg']= rn_df['rn'].apply(lambda x: x.split('_')[0])


There were 70 compounds in this seed set...
There were 4315 compounds produced in this expansion!


In [32]:
# for each compound, store a set of reacions that will be masked for each expansion
cpd_to_remove = [x for x in list(ne_cpds) if x not in seed_set]

rxns_to_remove = {}
for c in cpd_to_remove:
    r = metabolism.network[metabolism.network.cid == c][["rn","direction"]].drop_duplicates()
    rxns_to_remove[c] = list(zip(r["rn"].values,r["direction"].values))
    
rxns_to_remove = list(rxns_to_remove.values())    

In [33]:
len(rxns_to_remove)

4248

In [35]:
cx,rx = metabolism.run_expansions_reactionMasks_parallel(seed_set, rxns_to_remove)
del rx
# only keep network sizes
cx = [len(x) for x in cx]
results = pd.DataFrame({"cid":cpd_to_remove,"Scope after removal":cx})

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt

